In [1]:
import numpy as np
import random

import math
import time

In [2]:
import torch as th

In [3]:
device = th.device('cuda')

In [4]:
from gene_mat import revers_mat, trans_rev, trans_mat

In [5]:
n = 200
tmp = np.array([range(1,n + 1)])
mat_list = []
for _ in range(20):
    p1,p2,p3,p4 = [random.randint(0,n) for _ in range(4)]
    mat_list.append(trans_rev(n, p1, p2, p3, p4))
tmp_tensor = th.tensor(tmp, dtype = th.float, device = device)
mat_tensor = th.tensor(np.array(mat_list), dtype = th.float, device = device)
test = th.matmul(tmp_tensor, mat_tensor)

In [6]:
p1,p2,p3,p4

(120, 3, 97, 39)

In [7]:
# n = 6
# count = 2**n*math.factorial(n)
# for _ in range(10):
#     num = []
#     genes_5 = []
#     g_5 = [list(range(1, n + 1))]
#     test = th.tensor(g_5, dtype = th.float, device = device)
#     num.append((1,-1))
#     genes_5.append(g_5)
    
#     mat_op_list = [trans_rev, trans_mat, revers_mat]
#     for i in range(1000):
#         op_list = [mat_op_list[random.randint(0,2)](
#     n, *[random.randint(0,n) for _ in range(4)]) 
#            for _ in range(200)]
        
#         mat_tensor = th.tensor(op_list, dtype = th.float, device = device)
#         test = th.matmul(test, mat_tensor)
        
#         g_res = test.cpu().numpy().astype(dtype=np.int32).tolist()
#         for g_5 in g_res:
#             if g_5 not in genes_5:
#                 genes_5.append(g_5)
#         l = len(genes_5)
#         if l > num[-1][0]:
#             num.append((l, i))
#             if l%100 == 0 or i % 1000 == 0: 
#                 print(time.ctime(), num[-1])
#         if l >= count:
#             break
#     print(time.ctime(), num[-1])

In [8]:
n = 10
print(time.ctime())
test = th.tensor(np.diag(np.repeat(1,n)), 
                 dtype = th.float, device = device)
mat_op_list = [trans_rev, trans_mat, revers_mat]

print(time.ctime())
for _ in range(10):    
    op_list = np.array([mat_op_list[random.randint(0,2)](
    n, *[random.randint(0,n) for _ in range(4)]) 
           for _ in range(20)])
    mat_tensor = th.tensor(op_list, dtype = th.float, device = device)
    test = th.matmul(test, mat_tensor)
#     print(test[0])
#     print(mat_tensor[0])
#     mat_tensor = mat_tensor[th.randperm(mat_tensor.shape[0])]
print(time.ctime())

Wed Nov  3 14:57:48 2021
Wed Nov  3 14:57:48 2021
Wed Nov  3 14:57:48 2021


In [9]:
th.matmul(test[0], test[0].T)

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], device='cuda:0')

In [10]:
test[0]

tensor([[ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.],
        [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]], device='cuda:0')

In [11]:
test[0].T

tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.]], device='cuda:0')

In [12]:
gene = th.tensor(np.arange(1, 11).reshape((1,10)), dtype = th.float, device = device)

In [13]:
gene

tensor([[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.]], device='cuda:0')

In [14]:
t1 = th.matmul(gene, test[0])

In [15]:
t1

tensor([[ 9.,  7.,  1., 10., -8.,  4., -5.,  2.,  3., -6.]], device='cuda:0')

In [16]:
t2 = th.matmul(t1, test[0].T)

In [17]:
t2

tensor([[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.]], device='cuda:0')

In [18]:
# p1,p2,p3,p4 = [random.randint(0,n) for _ in range(4)]
# for _ in range(2000):
#     op_type = random.randint(0,2)
#     if op_type == 0:
#         op_mat = trans_rev(n, p1, p2, p3, p4)
#     elif op_type == 1:
#         op_mat = trans_mat(n, p1, p2, p3, p4)
#     else:
#         op_mat = revers_mat(n, p1, p2)
#     op_list.append(op_mat)

---

In [19]:
from dcj_comp import dcj_dist

In [53]:
def rand_param(size, op_type):    
    p = [random.randint(0,size) for _ in range(4)]
    if op_type == 2: # reverse operation
        while p[0] == p[1]:
            p = [random.randint(0,size) for _ in range(4)]
    elif op_type == 0: # trans reverse
        while p[0] == p[1] or \
        (min(p[0], p[1]) == p[2] and (p[3] == 0 or p[3] % abs(p[0] - p[1]) == 0)) or \
        p[2] % (size - abs(p[0] - p[1]) + 1) == min(p[0], p[1]):
            p = [random.randint(0,size) for _ in range(4)]
    elif op_type == 1: # trans op
        p = sorted(p)
        while (p[0] == p[1] and p[2] == p[3]) or \
        (p[0] == p[1] and p[1] == p[2]) or \
        (p[1] == p[2] and p[2] == p[3]):
            p = sorted([random.randint(0,size) for _ in range(4)])
            
    return p

In [160]:
n, vol = 50, 2000
gene = th.tensor(np.arange(1, n+1).reshape((1,n)), dtype = th.float, device = device)
oriseq = gene.cpu().numpy().astype(np.int32).reshape(n,).tolist()

genome = []
genome.append(oriseq)

t_dist = [0 for _ in range(vol)]

In [161]:
test = th.tensor(np.diag(np.repeat(1,n)), 
                 dtype = th.float, device = device)
mat_op_list = [trans_rev, trans_mat, revers_mat]
rand_op = [1 for _ in range(vol)] #[random.randint(0,2) for _ in range(vol)]
param_op = [rand_param(n, op_type) for op_type in rand_op] #[[random.randint(0,n) for _ in range(4)] for _ in range(vol)]
t_dist = np.add(t_dist, [1 if x == 2 else 2 for x in rand_op])

# op_list = np.array([mat_op_list[random.randint(0,2)](
# n, *[random.randint(0,n) for _ in range(4)]) 
#        for _ in range(20)])
op_list = np.array([mat_op_list[op](n, *param) 
       for op, param in zip(rand_op, param_op)])


mat_tensor = th.tensor(op_list, dtype = th.float, device = device)
test = th.matmul(test, mat_tensor)
trans_seq = th.matmul(gene, test)

In [56]:
new_seq = trans_seq.cpu().numpy().astype(np.int32)
for x, d, o, op_param in zip(new_seq, t_dist, rand_op, param_op):
    if dcj_dist(oriseq, x[0])[-1] != d:
        print(x[0], '***\t', dcj_dist(oriseq, x[0])[-1], d, mat_op_list[o].__name__, op_param)
    if x[0].tolist() in genome:
        continue
    genome.append(x[0].tolist())

In [57]:
dcj_dist([1,2,3,4,5], [-1, -3, -2,  4,  5])

(4, 0, 2)

In [61]:
def generate_seq_pos(gene):
    if len(gene) == 1:
        return [[gene[0]]]
    res = []
    for i in range(len(gene)):
        g = gene[i]
        rest = gene[0:i] + gene[i+1:]
        tmp = generate_seq_pos(rest)
        res += [[g] + x for x in tmp] #+ [[-g] + x for x in tmp]
    return res

In [118]:
def generate_seq(gene):
    if len(gene) == 1:
        return [[gene[0]], [-gene[0]]]
    res = []
    for i in range(len(gene)):
        g = gene[i]
        rest = gene[0:i] + gene[i+1:]
        tmp = generate_seq(rest)
        res += [[g] + x for x in tmp] + [[-g] + x for x in tmp]
    return res

In [124]:
for x in generate_seq([1,2,3]):
    print(x, dcj_dist([1,2,3], x)[-1])

[1, 2, 3] 0
[1, 2, -3] 1
[1, -2, 3] 1
[1, -2, -3] 2
[1, 3, 2] 2
[1, 3, -2] 2
[1, -3, 2] 2
[1, -3, -2] 1
[-1, 2, 3] 1
[-1, 2, -3] 2
[-1, -2, 3] 2
[-1, -2, -3] 3
[-1, 3, 2] 3
[-1, 3, -2] 3
[-1, -3, 2] 3
[-1, -3, -2] 2
[2, 1, 3] 2
[2, 1, -3] 3
[2, -1, 3] 2
[2, -1, -3] 3
[2, 3, 1] 2
[2, 3, -1] 2
[2, -3, 1] 3
[2, -3, -1] 3
[-2, 1, 3] 2
[-2, 1, -3] 3
[-2, -1, 3] 1
[-2, -1, -3] 2
[-2, 3, 1] 3
[-2, 3, -1] 3
[-2, -3, 1] 2
[-2, -3, -1] 3
[3, 1, 2] 2
[3, 1, -2] 3
[3, -1, 2] 3
[3, -1, -2] 2
[3, 2, 1] 2
[3, 2, -1] 3
[3, -2, 1] 3
[3, -2, -1] 2
[-3, 1, 2] 2
[-3, 1, -2] 3
[-3, -1, 2] 3
[-3, -1, -2] 3
[-3, 2, 1] 3
[-3, 2, -1] 2
[-3, -2, 1] 2
[-3, -2, -1] 1


In [259]:
genome = []

In [260]:
n, vol = 100, 20000
gene = th.tensor(np.arange(1, n+1).reshape((1,n)), dtype = th.float, device = device)
oriseq = gene.cpu().numpy().astype(np.int32).reshape(n,).tolist()
genome.append(oriseq)

t_dist = [0 for _ in range(vol)]

test = th.tensor(np.diag(np.repeat(1,n)), 
                 dtype = th.float, device = device)

In [261]:
for _ in range(10):
    mat_op_list = [trans_rev, trans_mat, revers_mat]
    rand_op = [random.randint(0,2) for _ in range(vol)]
    param_op = [rand_param(n, op_type) for op_type in rand_op] #[[random.randint(0,n) for _ in range(4)] for _ in range(vol)]
    t_dist = np.add(t_dist, [1 if x == 2 else 2 for x in rand_op])

    op_list = np.array([mat_op_list[op](n, *param) 
           for op, param in zip(rand_op, param_op)])


    mat_tensor = th.tensor(op_list, dtype = th.float, device = device)
    test = th.matmul(test, mat_tensor)
    trans_seq = th.matmul(gene, test)
    new_seq = trans_seq.cpu().numpy().astype(np.int32)

    # tmp = np.unique(new_seq, axis=0).reshape((-1,n))
    # tmp.shape
    count = 0
    tmp = []
    for x, d, o, op_param in zip(new_seq, t_dist, rand_op, param_op):
        if dcj_dist(oriseq, x[0])[-1] != d:
    #         print(x[0], '***\t', dcj_dist(oriseq, x[0])[-1], d, mat_op_list[o].__name__, op_param)
            count += 1
            continue
        tmp.append(x[0])

    tmp = np.unique(tmp, axis = 0)
    if len(tmp) != 0:
        genome += tmp.tolist()
    print(count)

    genome = np.unique(genome, axis = 0).tolist()

0
15
37
81
134
210
308
410
513
666


In [325]:
def gen_random_seq(l, n, r_n):
    res = np.zeros((n, l), dtype = np.int32)
    for i in range(n):
        s = np.random.permutation(l) + 1
        s[np.random.randint(l, size = r_n)] *=-1
        res[i] = s
    return res

In [421]:
def gen_seqs(l,n):
    genes = np.zeros((n * (l+1), l), dtype = np.int32)
    for i in range(l+1):
        genes[i*n : (i+1)*n] = gen_random_seq(l,n,i)
    np.random.shuffle(genes)
    return np.reshape(genes, (-1,1,l))

In [422]:
l, n = 10, 100
tmp = gen_seqs(l,n)

In [423]:
tmp.shape

(1100, 1, 10)

In [424]:
def gen_op_mat(l, n):
    t_dist = [0 for _ in range(n)]
    
    mat_op_list = [trans_rev, trans_mat, revers_mat]
    rand_op = [random.randint(0,2) for _ in range(n)]
    param_op = [rand_param(l, op_type) for op_type in rand_op] #[[random.randint(0,n) for _ in range(4)] for _ in range(vol)]
    t_dist = np.add(t_dist, [1 if x == 2 else 2 for x in rand_op])

    op_list = np.array([mat_op_list[op](l, *param) 
           for op, param in zip(rand_op, param_op)])
    return op_list, t_dist

In [425]:
o, t = gen_op_mat(tmp.shape[-1], tmp.shape[0])

In [426]:
def gen_dataset(l,n):
    s = gen_seqs(l,n)
    o, t = gen_op_mat(s.shape[-1], s.shape[0])
    new_seq = np.matmul(s, o)
    
    return s, o, t

In [438]:
s, o, t = gen_dataset(4, 10)

In [439]:
tmp = np.matmul(s,o)

In [440]:
tmp.shape

(50, 1, 4)

In [443]:
_, o1, t = gen_dataset(4, 10)
_, o2, t = gen_dataset(4, 10)

In [444]:
o3 = np.matmul(o, o1)
o3 = np.matmul(o3, o2)

In [448]:
np.matmul(np.matmul(o, o1), o2)[:2]

array([[[-1,  0,  0,  0],
        [ 0,  0,  1,  0],
        [ 0,  0,  0, -1],
        [ 0,  1,  0,  0]],

       [[-1,  0,  0,  0],
        [ 0,  0, -1,  0],
        [ 0,  0,  0,  1],
        [ 0,  1,  0,  0]]])

In [449]:
np.matmul(np.matmul(o, o2), o1)[:2]

array([[[ 0,  0, -1,  0],
        [ 0,  0,  0, -1],
        [ 1,  0,  0,  0],
        [ 0,  1,  0,  0]],

       [[ 0,  0,  1,  0],
        [ 0, -1,  0,  0],
        [ 0,  0,  0,  1],
        [-1,  0,  0,  0]]])

In [450]:
np.matmul(np.matmul(o1, o), o2)[:2]

array([[[ 0,  0,  0,  1],
        [ 0,  0,  1,  0],
        [ 0, -1,  0,  0],
        [-1,  0,  0,  0]],

       [[-1,  0,  0,  0],
        [ 0,  0, -1,  0],
        [ 0,  0,  0,  1],
        [ 0,  1,  0,  0]]])

In [447]:
o2[:2]

array([[[0, 1, 0, 0],
        [0, 0, 1, 0],
        [1, 0, 0, 0],
        [0, 0, 0, 1]],

       [[1, 0, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 1, 0, 0]]])

In [442]:
o

array([[[ 0,  0,  0,  1],
        [ 0,  1,  0,  0],
        [ 0,  0,  1,  0],
        [ 1,  0,  0,  0]],

       [[ 0,  1,  0,  0],
        [ 1,  0,  0,  0],
        [ 0,  0,  1,  0],
        [ 0,  0,  0,  1]],

       [[ 0, -1,  0,  0],
        [-1,  0,  0,  0],
        [ 0,  0,  1,  0],
        [ 0,  0,  0,  1]],

       [[-1,  0,  0,  0],
        [ 0,  1,  0,  0],
        [ 0,  0,  1,  0],
        [ 0,  0,  0,  1]],

       [[ 1,  0,  0,  0],
        [ 0,  1,  0,  0],
        [ 0,  0,  1,  0],
        [ 0,  0,  0, -1]],

       [[ 0,  0,  1,  0],
        [ 0, -1,  0,  0],
        [-1,  0,  0,  0],
        [ 0,  0,  0,  1]],

       [[ 1,  0,  0,  0],
        [ 0, -1,  0,  0],
        [ 0,  0,  1,  0],
        [ 0,  0,  0,  1]],

       [[ 0,  0, -1,  0],
        [ 0, -1,  0,  0],
        [ 1,  0,  0,  0],
        [ 0,  0,  0,  1]],

       [[ 0,  0,  1,  0],
        [ 1,  0,  0,  0],
        [ 0,  1,  0,  0],
        [ 0,  0,  0,  1]],

       [[-1,  0,  0,  0],
        [ 0,  1,  0,

In [433]:
a = np.concatenate((s,tmp), axis = 1)

In [434]:
a.shape

(1100, 2, 10)

In [437]:
np.matmul(a, o).shape

(1100, 2, 10)